Construindo um grafo automaticamente
====================================



## Introdução



Neste notebook nós vamos dar o primeiro passo para construir nossa rede neural artificial. Neste primeiro passo, nós vamos criar uma classe que gera automaticamente o nosso `grafo computacional`. O grafo computacional é o grafo que representa todas as operações matemáticas que ocorreram ao se computar um certo valor $y$. O grafo computacional é um passo necessário pois será baseado nele que iremos computar os gradientes locais necessários para realizar o `backpropagation`.



## Importações



In [1]:
from funcoes import plota_grafo

## Código e discussão



### Primeiros passos



A base de tudo será uma classe chamada `Valor`. Vamos começar pelo básico!



In [2]:
class Valor:
    def __init__(self, data):
        self.data = data

    def __repr__(self):
        return f"Valor(data={self.data})"

Vamos testar nossa classe!



In [3]:
objeto1 = Valor(25)
print(objeto1)
print(objeto1.data)

Valor(data=25)
25


### Os dunders `__add__` e `__mul__`



Observe que não conseguimos adicionar nem multiplicar objetos criados com a classe `Valor`.



In [4]:
a = Valor(10)
b = Valor(5)

In [5]:
print(a + b)

TypeError: unsupported operand type(s) for +: 'Valor' and 'Valor'

In [6]:
print(a * b)

TypeError: unsupported operand type(s) for *: 'Valor' and 'Valor'

In [7]:
# O Python reporta erro porque ele não consegue realizar operações matemáticas entre duas classes

Ué&#x2026; porque não conseguimos? Não conseguimos pois o Python (ainda) não é vidente. Ele lá vai saber como adicionar ou multiplicar algo que você criou? Pra você parece óbvio que valores podem ser adicionados ou multiplicados, mas para o Python ele nem sabe o que significa a palavra `Valor`&#x2026;

Como sempre, temos que contar para o programa o que queremos que aconteça quando usarmos os operadores `+` e `*`. Quem faz isso são os dunders `__add__` e `__mul__`.



In [8]:
class Valor:
    def __init__(self, data):
        self.data = data

    def __repr__(self):
        return f"Valor(data={self.data})"

    def __add__(self, outro_valor):
        saida = Valor(self.data + outro_valor.data)
        return saida

    def __mul__(self, outro_valor):
        saida = Valor(self.data * outro_valor.data)
        return saida

Vamos testar!



In [9]:
a = Valor(10)
b = Valor(5)

print(a + b)
print(a * b)

Valor(data=15)
Valor(data=50)


### Registrando os progenitores



Nosso objetivo é construir um grafo computacional. Em um grafo computacional, um certo vértice pode ter um ou mais vértices progenitores (são seus pais/mães). Nós não podemos perder essa informação quando formos construir um grafo, então precisamos incluir essa informação na nossa classe.



In [10]:
class Valor:
    def __init__(self, data, progenitor=()):
        self.data = data
        self.progenitor = progenitor

    def __repr__(self):
        return f"Valor(data={self.data})"

    def __add__(self, outro_valor):
        data = self.data + outro_valor.data
        progenitor = (self, outro_valor)
        saida = Valor(data, progenitor)
        return saida

    def __mul__(self, outro_valor):
        data = self.data * outro_valor.data
        progenitor = (self, outro_valor)
        saida = Valor(data, progenitor)
        return saida

Vamos testar!



In [11]:
a = Valor(10)
b = Valor(5)

print(a.data)
c = a + b
print(c.progenitor)

10
(Valor(data=10), Valor(data=5))


### Registrando o operador mãe



Em um grafo computacional, um vértice pode ter um operador mãe. O operador mãe é o operador que foi usado para gerar o vértice.



In [12]:
class Valor:
    def __init__(self, data, progenitor=(), operador_mae=""):
        self.data = data
        self.progenitor = progenitor
        self.operador_mae = operador_mae

    def __repr__(self):
        return f"Valor(data={self.data})"

    def __add__(self, outro_valor):
        data = self.data + outro_valor.data
        progenitor = (self, outro_valor)
        operador_mae = "+"
        saida = Valor(data, progenitor, operador_mae)
        return saida

    def __mul__(self, outro_valor):
        data = self.data * outro_valor.data
        progenitor = (self, outro_valor)
        operador_mae = "*"
        saida = Valor(data, progenitor, operador_mae)
        return saida

Vamos testar!



In [13]:
a = Valor(10)
b = Valor(5)

c = a + b
d = c * a

print(c)
print(c.progenitor)
print(c.operador_mae)

Valor(data=15)
(Valor(data=10), Valor(data=5))
+


### Plotando o primeiro grafo



Vamos plotar nosso primeiro grafo!



In [14]:
a = Valor(2)
b = Valor(-3)
c = Valor(10)

d = a * b
e = d + c

grafo1 = plota_grafo(e)
print(grafo1)

digraph {
	graph [rankdir=LR]
	2133184830480 [label="{ data -3.0000 }" shape=record]
	2133184827936 [label="{ data -6.0000 }" shape=record]
	"2133184827936*" [label="*"]
	"2133184827936*" -> 2133184827936
	2133184828560 [label="{ data 2.0000 }" shape=record]
	2133185203648 [label="{ data 4.0000 }" shape=record]
	"2133185203648+" [label="+"]
	"2133185203648+" -> 2133185203648
	2133184827888 [label="{ data 10.0000 }" shape=record]
	2133184828560 -> "2133184827936*"
	2133184827936 -> "2133185203648+"
	2133184830480 -> "2133184827936*"
	2133184827888 -> "2133185203648+"
}



### Registrando o rótulo



Nosso grafo seria mais legível se tivéssemos rótulos indicando o que é cada vértice. Vamos incluir essa informação na nossa classe.



In [15]:
class Valor:
    def __init__(self, data, progenitor=(), operador_mae="", rotulo=""):
        self.data = data
        self.progenitor = progenitor
        self.operador_mae = operador_mae
        self.rotulo = rotulo

    def __repr__(self):
        return f"Valor(data={self.data})"

    def __add__(self, outro_valor):
        data = self.data + outro_valor.data
        progenitor = (self, outro_valor)
        operador_mae = "+"
        saida = Valor(data, progenitor, operador_mae)
        return saida

    def __mul__(self, outro_valor):
        data = self.data * outro_valor.data
        progenitor = (self, outro_valor)
        operador_mae = "*"
        saida = Valor(data, progenitor, operador_mae)
        return saida

Vamos testar!



In [16]:
a = Valor(2, rotulo="a")
b = Valor(-3, rotulo="b")
c = Valor(10, rotulo="c")

d = a * b
e = d + c

d.rotulo = "d"
e.rotulo = "e"

grafo2 = plota_grafo(e)
print(grafo2)

digraph {
	graph [rankdir=LR]
	2133185925184 [label="{ b | data -3.0000 }" shape=record]
	2133185927296 [label="{ e | data 4.0000 }" shape=record]
	"2133185927296+" [label="+"]
	"2133185927296+" -> 2133185927296
	2133185928112 [label="{ d | data -6.0000 }" shape=record]
	"2133185928112*" [label="*"]
	"2133185928112*" -> 2133185928112
	2133185928640 [label="{ c | data 10.0000 }" shape=record]
	2133185927632 [label="{ a | data 2.0000 }" shape=record]
	2133185928640 -> "2133185927296+"
	2133185925184 -> "2133185928112*"
	2133185928112 -> "2133185927296+"
	2133185927632 -> "2133185928112*"
}



### Refazendo o grafo que fizemos na aula anterior



Na aula anterior nós fizemos um grafo computacional para aprender como funciona o backpropagation. Vamos refazer ele aqui!



In [17]:
x1 = Valor(60, rotulo = 'x1')
x2 = Valor(24, rotulo = 'x2')
w1 = Valor(10, rotulo = 'w1')
w2 = Valor(5, rotulo = 'w2')
w3 = Valor(2, rotulo = 'w3')
b = Valor(7, rotulo = 'y')

s1 = x1 * w1
s1.rotulo = 's1'

s2 = x2 * w2
s2.rotulo = 's2'

n = s1 + s2
n.rotulo = 'n'

k = n + b
k.rotulo = 'k'

y = w3 * k

grafo_aula = plota_grafo(y)
print(grafo_aula)

digraph {
	graph [rankdir=LR]
	2133184828416 [label="{ s1 | data 600.0000 }" shape=record]
	"2133184828416*" [label="*"]
	"2133184828416*" -> 2133184828416
	2133184828992 [label="{ w3 | data 2.0000 }" shape=record]
	2133184827456 [label="{ n | data 720.0000 }" shape=record]
	"2133184827456+" [label="+"]
	"2133184827456+" -> 2133184827456
	2133184830528 [label="{ s2 | data 120.0000 }" shape=record]
	"2133184830528*" [label="*"]
	"2133184830528*" -> 2133184830528
	2133184830576 [label="{ w2 | data 5.0000 }" shape=record]
	2133184846480 [label="{ k | data 727.0000 }" shape=record]
	"2133184846480+" [label="+"]
	"2133184846480+" -> 2133184846480
	2133184720032 [label="{ x2 | data 24.0000 }" shape=record]
	2133184719552 [label="{ w1 | data 10.0000 }" shape=record]
	2133184829712 [label="{ y | data 7.0000 }" shape=record]
	2133184845616 [label="{  | data 1454.0000 }" shape=record]
	"2133184845616*" [label="*"]
	"2133184845616*" -> 2133184845616
	2133184720704 [label="{ x1 | data 60.0000 }" s

## Conclusão

Nesse experimento, foi realizada a prática de construir grafos, a partir de classes. Foi constatado que os operadores aritméticos, como de adição, subtração, multiplicação e divisão não funcionam para objetos que têm atribuição em classe. Isso ocorre porque o Python não interpreta esses objetos como números. Para aplicar as operações de soma e multiplicação, por exemplo, é preciso utilizar seus respectivos dunders `__add__` e `__mul__`, como foi feito para a classe Valor. Um grafo é constituído por um conjunto de nós ou vértices que são conectados por ligações ou arestas. Operadores quando aplicados em pares de nós, resultam em um outro valor. A fim de não perder a informação sobre os pares de dados que geraram o dado seguinte, pode ser definido um método, neste caso, `.progenitor` para identificar essa informação.

Os operadores aplicados em grafos podem ser representados por um objeto que armazena a propriedade na forma de string. Outro ponto importante na construção de grafos é o rótulo referenciando cada valor do vértice, isso pode ser feito atribuindo o método `.rotulo` na classe, no formato de string, assim como foi feito para os operadores.

Portanto, é possível manipular grafos de vários tamanhos e com diversas operações, estruturando as ações a serem realizadas em uma classe.

## Playground

